In [1]:
# Multiple Participants -- Approach 1: Model Template
# Train & Test on Distinct Groups of Participants
# Approach 1: don't use any test participant data

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

In [3]:
# Key Features: op_num_people, of_pose_distance, of_gaze_distance, of_confidence, ros_mistakes_session, ros_ts_robot_talked, timestamp
use_key_features = True

# Leave One Out Run: test on each participant & train on remaining
leave_one_out_run = False

# File Name for Model Results
output_file = 'multi1_knn_results.csv'

# 1 -- Load Data

In [4]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [5]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [6]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [7]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [8]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [9]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [10]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [11]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [12]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [13]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [14]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [15]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [16]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [17]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [18]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for training
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i].copy())
    
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i].copy(), split_size)
            train_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i].copy())
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)

    y_train = train_data['engagement']
    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])
    y_test = test_data['engagement']
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    # Sort Training Data Together 
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [19]:
# MODEL HERE
# Multiple Participants Approach 1: 
# No hyperparameter search -- instead use most general hyperparameters & boosting

# Inputs: X_train, y_train, X_test, y_test

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn import neighbors

def model(X_train, y_train, X_test, y_test):
    clf = neighbors.KNeighborsClassifier()
    clf.fit(X_train, y_train)

    scores = clf.predict_proba(X_test.values)[:,1]
    pred = [round(value) for value in scores]   
    
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    try:
        auc = roc_auc_score(y_test, scores)
    except:
        auc = np.nan
    results['auc'] = auc

    precision = metrics.precision_score(y_test, pred, average=None)
    recall = metrics.recall_score(y_test, pred, average=None)
    f1 = metrics.f1_score(y_test, pred, average=None)

    results['precision_0'], results['precision_1'] = precision[0], precision[1]
    results['recall_0'], results['recall_1'] = recall[0], recall[1]
    results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [20]:
all_results = []
for n in range(1, len(part)):    
    if leave_one_out_run:
        if n != len(part)-1:
            continue
    
    comb = combinations(part, n)
    print()
    
    for c in comb:
        print()
        new_results = {}
        
        # Determine Train - Test Participants
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part, '\tTest:', test_part)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split(train_part, test_part, 0)
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        
        # Evaluate on Test  
        results = model(X_train, y_train, X_test, y_test)
        print(results)
        results['train_participants'] = ", ".join(str(x) for x in train_part)
        results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(results)



Train: [5] 	Test: [7, 9, 11, 12, 17, 18]
{'recall_0': 0.505938471174926, 'f1_1': 0.8011104216336261, 'auc': 0.7230125846607695, 'accuracy': 0.7299974284624866, 'precision_1': 0.7493859295377058, 'f1_0': 0.5797297483816649, 'precision_0': 0.678721478895999, 'recall_1': 0.8605046026044005}

Train: [7] 	Test: [5, 9, 11, 12, 17, 18]
{'recall_0': 0.42023467027697864, 'f1_1': 0.7849500546016093, 'auc': 0.6740267793177979, 'accuracy': 0.6959746164046221, 'precision_1': 0.740260560799626, 'f1_0': 0.481412669434706, 'precision_0': 0.5634383390689407, 'recall_1': 0.8353820266377663}

Train: [9] 	Test: [5, 7, 11, 12, 17, 18]
{'recall_0': 0.6800535892323031, 'f1_1': 0.75598144624531, 'auc': 0.7226898404906188, 'accuracy': 0.6906466456769194, 'precision_1': 0.8280878561623196, 'f1_0': 0.5775331736938891, 'precision_0': 0.5018739509323769, 'recall_1': 0.6954266192410972}

Train: [11] 	Test: [5, 7, 9, 12, 17, 18]
{'recall_0': 0.6527450055057417, 'f1_1': 0.8098938415883992, 'auc': 0.7723690698607002

{'recall_0': 0.4918249305460673, 'f1_1': 0.8145922146900552, 'auc': 0.7132758142012843, 'accuracy': 0.735327651569136, 'precision_1': 0.7852330908111022, 'f1_0': 0.537678309143867, 'precision_0': 0.5929606852624643, 'recall_1': 0.8462320313634384}

Train: [5, 7, 11] 	Test: [9, 12, 17, 18]
{'recall_0': 0.42070492447237795, 'f1_1': 0.7594554030874786, 'auc': 0.6758550587464762, 'accuracy': 0.6778001148765077, 'precision_1': 0.6858819201053385, 'f1_0': 0.5122173913043478, 'precision_0': 0.6546091015169195, 'recall_1': 0.8507097053101809}

Train: [5, 7, 12] 	Test: [9, 11, 17, 18]
{'recall_0': 0.4106956865280322, 'f1_1': 0.7670494052205288, 'auc': 0.6611746317802343, 'accuracy': 0.6782098541093401, 'precision_1': 0.7132721712538226, 'f1_0': 0.47983718685745463, 'precision_0': 0.5769715929158888, 'recall_1': 0.8295969594113941}

Train: [5, 7, 17] 	Test: [9, 11, 12, 18]
{'recall_0': 0.40501082816817263, 'f1_1': 0.7759392365573292, 'auc': 0.6455205748044209, 'accuracy': 0.6836038696920016, 'pr

{'recall_0': 0.6430697087203535, 'f1_1': 0.8128706514760771, 'auc': 0.767955563983547, 'accuracy': 0.7500799302426973, 'precision_1': 0.8251264718227407, 'f1_0': 0.6238708689661205, 'precision_0': 0.6057851590706367, 'recall_1': 0.8009735803739921}

Train: [9, 17, 18] 	Test: [5, 7, 11, 12]
{'recall_0': 0.6446464933465927, 'f1_1': 0.8083866134378752, 'auc': 0.7540750666695654, 'accuracy': 0.739482900556726, 'precision_1': 0.8399624645439229, 'f1_0': 0.5931978124931913, 'precision_0': 0.5493543179983857, 'recall_1': 0.7790987498021839}

Train: [11, 12, 17] 	Test: [5, 7, 9, 18]
{'recall_0': 0.6760590525382569, 'f1_1': 0.7837079854420894, 'auc': 0.7609066902866879, 'accuracy': 0.7282386485904885, 'precision_1': 0.8132900846459754, 'f1_0': 0.6345054627016858, 'precision_0': 0.5977642207150404, 'recall_1': 0.7562023590654631}

Train: [11, 12, 18] 	Test: [5, 7, 9, 17]
{'recall_0': 0.6734339569160998, 'f1_1': 0.7909413222430777, 'auc': 0.7819740538710075, 'accuracy': 0.7504393415734286, 'preci

{'recall_0': 0.49186440677966103, 'f1_1': 0.8008966083217992, 'auc': 0.6764283711656055, 'accuracy': 0.7031158679135009, 'precision_1': 0.8450205484763111, 'f1_0': 0.41660984100778814, 'precision_0': 0.36132727385917945, 'recall_1': 0.7611519836096108}

Train: [7, 9, 12, 18] 	Test: [5, 11, 17]
{'recall_0': 0.5612527519352318, 'f1_1': 0.8192254795512125, 'auc': 0.7355534012291229, 'accuracy': 0.7435091611206968, 'precision_1': 0.8207820840102111, 'f1_0': 0.558654084049058, 'precision_0': 0.5560793695468618, 'recall_1': 0.8176747680836922}

Train: [7, 9, 17, 18] 	Test: [5, 11, 12]
{'recall_0': 0.5304915514592934, 'f1_1': 0.8266652647074286, 'auc': 0.7191202650177305, 'accuracy': 0.7441359641464408, 'precision_1': 0.8372514043822049, 'f1_0': 0.5115917339456337, 'precision_0': 0.49399227578314975, 'recall_1': 0.8163434831139839}

Train: [7, 11, 12, 17] 	Test: [5, 9, 18]
{'recall_0': 0.4617841533116665, 'f1_1': 0.7698625006687712, 'auc': 0.6532965841713266, 'accuracy': 0.6826983365913031, '

{'recall_0': 0.5869136775719354, 'f1_1': 0.8455007674597084, 'auc': 0.7551070692971422, 'accuracy': 0.7742500700869077, 'precision_1': 0.8486351162678735, 'f1_0': 0.5810353798126952, 'precision_0': 0.5752736638763684, 'recall_1': 0.842389486260454}

Train: [7, 9, 12, 17, 18] 	Test: [5, 11]
{'recall_0': 0.46114615487853977, 'f1_1': 0.8258710061950258, 'auc': 0.6846948874290679, 'accuracy': 0.7317473567056205, 'precision_1': 0.8506213259915172, 'f1_0': 0.4161569672379338, 'precision_0': 0.3791657470757007, 'recall_1': 0.8025202709817825}

Train: [7, 11, 12, 17, 18] 	Test: [5, 9]
{'recall_0': 0.44609913314069793, 'f1_1': 0.74880810488677, 'auc': 0.6477771018006333, 'accuracy': 0.6667419664539448, 'precision_1': 0.7017592851158894, 'f1_0': 0.5050327126321088, 'precision_0': 0.581907799362134, 'recall_1': 0.8026189715745768}

Train: [9, 11, 12, 17, 18] 	Test: [5, 7]
{'recall_0': 0.7184140447775915, 'f1_1': 0.8121846125486762, 'auc': 0.7855420776856149, 'accuracy': 0.7534408179315769, 'preci

In [21]:
results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
results = results.append(all_results, ignore_index=True, sort=True)

In [22]:
results.head()

,accuracy,auc,f1_0,f1_1,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,0.729997,0.723013,0.579730,0.801110,0.678721,0.749386,0.505938,0.860505,"7, 9, 11, 12, 17, 18",5
1,0.695975,0.674027,0.481413,0.784950,0.563438,0.740261,0.420235,0.835382,"5, 9, 11, 12, 17, 18",7
2,0.690647,0.722690,0.577533,0.755981,0.501874,0.828088,0.680054,0.695427,"5, 7, 11, 12, 17, 18",9
3,0.757972,0.772369,0.667033,0.809894,0.681960,0.799897,0.652745,0.820143,"5, 7, 9, 12, 17, 18",11
4,0.726211,0.754999,0.624852,0.784449,0.592476,0.809877,0.660971,0.760569,"5, 7, 9, 11, 17, 18",12


In [23]:
results.to_csv(output_file, index=False)